# Algorithms 4 week 4: The 2-SAT (satisfiability) problem

In [32]:
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    from google.colab import files
    uploaded = files.upload()
    data = uploaded['2sat2.txt'].decode('ascii')
else:
    uploaded_edges = open('./2sat1.txt', 'r')
    data = uploaded_edges.read()

In [14]:
# Test case, uncomment to try it instead of the file

# Expected answer: False (not satisfiable)
# data = """20
# -3 -10
# -9 -20
# -5 4
# -9 -9
# -12 2
# -13 -17
# 19 2
# -6 -5
# -10 16
# 5 19
# 12 -10
# -12 8
# 12 12
# -16 -16
# 3 -19
# 10 -8
# -18 -19
# 17 -5
# 3 20
# -15 -10
# """

# Expected answer: True (satisfiable)
data="""40
27 11
-40 -34
29 -31
-35 17
-5 -1
-31 4
5 36
-23 20
-11 2
-24 -10
-15 35
-23 2
34 -13
-24 9
-14 -22
34 16
24 21
-11 27
-11 -14
15 -22
19 22
-23 -21
-26 -37
-40 27
-5 31
-21 9
-21 36
2 -9
10 24
1 5
-29 -30
1 -23
5 -11
37 -22
38 17
16 -10
25 6
19 -30
40 16
-24 9
"""

In [33]:
n = int(data.split('\n')[0])
clauses = data.split('\n')[1:-1]
clauses = [(int(line.split(' ')[0]), int(line.split(' ')[1])) for line in clauses]

clauses = [(x > 0, abs(x)-1, y > 0, abs(y)-1) for x, y in clauses]
assert n == len(clauses)
print(clauses[:3])

[(False, 16807, True, 75249), (True, 43658, True, 8930), (False, 27544, False, 50878)]


## Papadimitriou's local search algorithm

In [34]:
from collections import defaultdict

bit_to_clauses_idx = defaultdict(set)
for i, clause in enumerate(clauses):
    bit_to_clauses_idx[clause[1]].add(i)
    bit_to_clauses_idx[clause[3]].add(i)

In [35]:
from math import log2
import time
import numpy as np
from random import choice

def initialize_assignment(n):
    """Return an array of n random booleans"""
    return np.random.randint(2, size=n, dtype=bool)


def check_clause(assignment, clause):
    return ((not (assignment[clause[1]] ^ clause[0])) or (not (assignment[clause[3]] ^ clause[2])))


def get_clauses_satisfaction(assignment, clauses):
    satisfied_clauses, unsatisfied_clauses = set(), set()
    for i, clause in enumerate(clauses):
        if check_clause(assignment, clause):
            satisfied_clauses.add(i)
        else:
            unsatisfied_clauses.add(i)
    return satisfied_clauses, unsatisfied_clauses


def flip_unsatisfying_bit(assignment, clauses, satisfied_clauses, unsatisfied_clauses):
    """Flip a bit that belongs to an unsatisfied clause."""
    unsatisfied_clause_idx = unsatisfied_clauses.pop()
    satisfied_clauses.add(unsatisfied_clause_idx)
    
    flip_clause = clauses[unsatisfied_clause_idx]
    flipped_bit_idx = flip_clause[1+2*np.random.randint(2)]
    affected_clauses_idx = [i for i in bit_to_clauses_idx[flipped_bit_idx] 
                            if i != unsatisfied_clause_idx]
    assignment[flipped_bit_idx] = (not assignment[flipped_bit_idx])

    for i in affected_clauses_idx:
        if i in unsatisfied_clauses:  
            unsatisfied_clauses.remove(i)
            satisfied_clauses.add(i)
        else:
            if not check_clause(assignment, clauses[i]):
                satisfied_clauses.remove(i)
                unsatisfied_clauses.add(i)

    return assignment, satisfied_clauses, unsatisfied_clauses
        

def evaluate_satisfiability(clauses):
    """Return True if the set of clauses is satisfiable, and False otherwise. """
    n = len(clauses)
    print(n)
    outer_iterations = int(log2(n))
    inner_iterations = n #2*(n**2)
    
    t0 = time.time()
    for it in range(outer_iterations):
        print(f"{it}-th outer iteration, {time.time()-t0:.1f}s elapsed")
        assignment = initialize_assignment(n)
        satisfied_clauses, unsatisfied_clauses = get_clauses_satisfaction(assignment, clauses)
        
        for it_inner in range(inner_iterations):
            if len(unsatisfied_clauses) == 0:
                return True
            assignment, satisfied_clauses, unsatisfied_clauses = \
                flip_unsatisfying_bit(assignment, clauses, satisfied_clauses, unsatisfied_clauses)
    return False

evaluate_satisfiability(clauses)

100000
0-th outer iteration, 0.0s elapsed
1-th outer iteration, 1.9s elapsed
2-th outer iteration, 3.8s elapsed
3-th outer iteration, 5.6s elapsed
4-th outer iteration, 7.5s elapsed
5-th outer iteration, 9.6s elapsed
6-th outer iteration, 11.7s elapsed
7-th outer iteration, 13.9s elapsed
8-th outer iteration, 16.1s elapsed
9-th outer iteration, 18.2s elapsed
10-th outer iteration, 20.1s elapsed
11-th outer iteration, 22.0s elapsed
12-th outer iteration, 23.9s elapsed
13-th outer iteration, 25.7s elapsed
14-th outer iteration, 27.5s elapsed
15-th outer iteration, 29.4s elapsed


False